# Combining logic programming with LLMs for visualization recommendation 
<small>Author : Ivan Khodakov</small>

#### Part 1
- Use of an LLM to pick two columns to visualize from the dataframe. 
- Generation of chart recommendations using Draco.

I used the templates given in https://dig.cmu.edu/draco2/applications/design_space_exploration.html

In [14]:
# Display utilities
import json

import numpy as np
from IPython.display import Markdown, display


# Handles serialization of common numpy datatypes
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)


def md(markdown: str):
    display(Markdown(markdown))


def pprint(obj):
    md(f"```json\n{json.dumps(obj, indent=2, cls=NpEncoder)}\n```")

In [15]:
import altair as alt
import pandas as pd
from vega_datasets import data as vega_data

import draco as drc

# Loading data to be explored
df: pd.DataFrame = vega_data.seattle_weather()
df.head()

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


In [ ]:
import openai

# Configure your OpenAI API Key
openai.api_key = "YOUR_OPENAI_API_KEY"
print("Available columns:", df.columns.tolist())

column_info = df.dtypes.to_string()

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an assistant that helps visualize data."},
        {"role": "user", "content": f"Here are the columns of a DataFrame with their types:\n{column_info}\n\n"
                                   "I want you to choose 2 relevant columns (in terms of visualization) to visualize. Return only a list with the names of the 2 selected columns without any comments"}
    ]
)
columns = response['choices'][0]['message']['content'].strip()
# Display the LLM's response
print("LLM's response:", response['choices'][0]['message']['content'].strip())


Available columns: ['date', 'precipitation', 'temp_max', 'temp_min', 'wind', 'weather']
LLM's response: ['precipitation', 'temp_max']


In [17]:
data_schema = drc.schema_from_dataframe(df)
pprint(data_schema)

```json
{
  "number_rows": 1461,
  "field": [
    {
      "name": "date",
      "type": "datetime",
      "unique": 1461,
      "entropy": 7287
    },
    {
      "name": "precipitation",
      "type": "number",
      "unique": 111,
      "entropy": 2422,
      "min": 0,
      "max": 55,
      "std": 6
    },
    {
      "name": "temp_max",
      "type": "number",
      "unique": 67,
      "entropy": 3934,
      "min": -1,
      "max": 35,
      "std": 7
    },
    {
      "name": "temp_min",
      "type": "number",
      "unique": 55,
      "entropy": 3596,
      "min": -7,
      "max": 18,
      "std": 5
    },
    {
      "name": "wind",
      "type": "number",
      "unique": 79,
      "entropy": 3950,
      "min": 0,
      "max": 9,
      "std": 1
    },
    {
      "name": "weather",
      "type": "string",
      "unique": 5,
      "entropy": 1201,
      "freq": 714
    }
  ]
}
```

In [18]:
data_schema_facts = drc.dict_to_facts(data_schema)
pprint(data_schema_facts)

```json
[
  "attribute(number_rows,root,1461).",
  "entity(field,root,0).",
  "attribute((field,name),0,date).",
  "attribute((field,type),0,datetime).",
  "attribute((field,unique),0,1461).",
  "attribute((field,entropy),0,7287).",
  "entity(field,root,1).",
  "attribute((field,name),1,precipitation).",
  "attribute((field,type),1,number).",
  "attribute((field,unique),1,111).",
  "attribute((field,entropy),1,2422).",
  "attribute((field,min),1,0).",
  "attribute((field,max),1,55).",
  "attribute((field,std),1,6).",
  "entity(field,root,2).",
  "attribute((field,name),2,temp_max).",
  "attribute((field,type),2,number).",
  "attribute((field,unique),2,67).",
  "attribute((field,entropy),2,3934).",
  "attribute((field,min),2,-1).",
  "attribute((field,max),2,35).",
  "attribute((field,std),2,7).",
  "entity(field,root,3).",
  "attribute((field,name),3,temp_min).",
  "attribute((field,type),3,number).",
  "attribute((field,unique),3,55).",
  "attribute((field,entropy),3,3596).",
  "attribute((field,min),3,-7).",
  "attribute((field,max),3,18).",
  "attribute((field,std),3,5).",
  "entity(field,root,4).",
  "attribute((field,name),4,wind).",
  "attribute((field,type),4,number).",
  "attribute((field,unique),4,79).",
  "attribute((field,entropy),4,3950).",
  "attribute((field,min),4,0).",
  "attribute((field,max),4,9).",
  "attribute((field,std),4,1).",
  "entity(field,root,5).",
  "attribute((field,name),5,weather).",
  "attribute((field,type),5,string).",
  "attribute((field,unique),5,5).",
  "attribute((field,entropy),5,1201).",
  "attribute((field,freq),5,714)."
]
```

In [19]:
from draco.renderer import AltairRenderer

input_spec_base = data_schema_facts + [
    "entity(view,root,v0).",
    "entity(mark,v0,m0).",
]
d = drc.Draco()
renderer = AltairRenderer()

In [20]:
def recommend_charts(
    spec: list[str], draco: drc.Draco, num: int = 5, display_chart = True
) -> dict[str, tuple[list[str], dict]]:
    # Dictionary to store the generated recommendations, keyed by chart name
    chart_specs = []
    cost_draco = []
    for i, model in enumerate(draco.complete_spec(spec, num)):
        spec = drc.answer_set_to_dict(model.answer_set)
        chart_specs.append((drc.dict_to_facts(spec), spec))
        cost_draco.append(model.cost[0])
        if display_chart :
            print(f"COST: {model.cost}")
            chart = renderer.render(spec=spec, data=df)
            # Adjust column-faceted chart size
            if (
                isinstance(chart, alt.FacetChart)
                and chart.facet.column is not alt.Undefined
            ):
                chart = chart.configure_view(continuousWidth=130, continuousHeight=130)
            display(chart)

    return chart_specs, cost_draco

In [21]:
col1, col2 = columns.replace("'", "").replace("[", "").replace("]", "").split(",")  # Nettoyage de la chaîne
input_spec = input_spec_base + [
    # Ajouter la première colonne
    f"entity(encoding,m0,e0).",
    f"attribute((encoding,field),e0,{col1.strip()}).",

    # Ajouter la deuxième colonne
    f"entity(encoding,m0,e1).",
    f"attribute((encoding,field),e1,{col2.strip()}).",
]
print(input_spec)
recommendations, cost_draco = recommend_charts(spec=input_spec, draco=d, num=5)

['attribute(number_rows,root,1461).', 'entity(field,root,0).', 'attribute((field,name),0,date).', 'attribute((field,type),0,datetime).', 'attribute((field,unique),0,1461).', 'attribute((field,entropy),0,7287).', 'entity(field,root,1).', 'attribute((field,name),1,precipitation).', 'attribute((field,type),1,number).', 'attribute((field,unique),1,111).', 'attribute((field,entropy),1,2422).', 'attribute((field,min),1,0).', 'attribute((field,max),1,55).', 'attribute((field,std),1,6).', 'entity(field,root,2).', 'attribute((field,name),2,temp_max).', 'attribute((field,type),2,number).', 'attribute((field,unique),2,67).', 'attribute((field,entropy),2,3934).', 'attribute((field,min),2,-1).', 'attribute((field,max),2,35).', 'attribute((field,std),2,7).', 'entity(field,root,3).', 'attribute((field,name),3,temp_min).', 'attribute((field,type),3,number).', 'attribute((field,unique),3,55).', 'attribute((field,entropy),3,3596).', 'attribute((field,min),3,-7).', 'attribute((field,max),3,18).', 'attrib

alt.Chart(...)

COST: [14]


alt.Chart(...)

COST: [14]


alt.Chart(...)

COST: [14]


alt.Chart(...)

COST: [14]


alt.Chart(...)

#### Part 2 :
- assign a score to a Vega-Lite chart
- check if the columns picked by the LLM resulted in the best chart or if it's possible to get a better solution by checking all pairs of columns.

In [22]:
def score_with_llm(df, recommendations) :
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant tasked with evaluating the relevance of chart recommendations in terms of visualization."},
            {"role": "user", "content": f"I will give you 5 chart specifications derived from the 'seattle_weather' dataset ({df.head()}). Please evaluate the relevance (in terms of visualization) of each chart specification on a scale from 0 to 10, where 0 means 'not relevant at all' and 10 means 'extremely relevant'. Here are the 5 chart recommendations: {recommendations}. Please return only a list with the scores. Do not return a dictionnary and make sure to return 5 score."}
        ]
    )


    score_llm = response['choices'][0]['message']['content'].strip()
    # Afficher la réponse du LLM
    print("LLM's response :", response['choices'][0]['message']['content'].strip())
    return eval(score_llm)

score_llm = score_with_llm(df, recommendations)

LLM's response : [6, 7, 8, 7, 6]


In [23]:
def final_score(score_llm, cost_draco) :
    cost_draco = np.array(cost_draco)
    score_draco = (np.max(cost_draco)-cost_draco+1)/(np.max(cost_draco)-np.min(cost_draco)+1)
    score_llm = np.array(score_llm)
    score_llm = score_llm/10
    return (score_llm+score_draco)/2

final_score(score_llm,cost_draco)

array([0.8 , 0.85, 0.9 , 0.85, 0.8 ])

In [24]:
def return_best_chart(score_llm, cost_draco, recommendations,df):
    score = final_score(score_llm,cost_draco)
    index = np.argmax(score)
    print('best_score : ', score[index])
    best_spec= recommendations[index][1]

    chart = renderer.render(spec=best_spec, data=df)
    # Adjust column-faceted chart size
    if (
        isinstance(chart, alt.FacetChart)
        and chart.facet.column is not alt.Undefined
    ):
        chart = chart.configure_view(continuousWidth=130, continuousHeight=130)
    display(chart)

return_best_chart(score_llm,cost_draco,recommendations,df)

best_score :  0.9


alt.Chart(...)

In [25]:
import itertools

column_pairs = list(itertools.combinations(df.columns, 2))
all_cost_draco = []
all_recommendations = []
all_score_llm= []
for columns in column_pairs :
    col1 = columns[0]
    col2 = columns[1]
    input_spec = input_spec_base + [
        # Ajouter la première colonne
        f"entity(encoding,m0,e0).",
        f"attribute((encoding,field),e0,{col1.strip()}).",

        # Ajouter la deuxième colonne
        f"entity(encoding,m0,e1).",
        f"attribute((encoding,field),e1,{col2.strip()}).",
    ]
    recommendations, cost_draco = recommend_charts(spec=input_spec, draco=d, num=5, display_chart=False)
    score_llm = score_with_llm(df, recommendations)
    all_cost_draco.extend(cost_draco)
    all_recommendations.extend(recommendations)
    all_score_llm.extend(score_llm)
return_best_chart(all_score_llm, all_cost_draco, all_recommendations, df)


LLM's response : [8, 7, 9, 6, 5]
LLM's response : [7, 6, 8, 5, 9]
LLM's response : [8, 7, 9, 5, 6]
LLM's response : [8, 6, 10, 7, 9]
LLM's response : [8, 7, 9, 6, 7]
LLM's response : [7, 6, 8, 5, 9]
LLM's response : [7, 6, 8, 5, 9]
LLM's response : [7, 5, 8, 6, 9]
LLM's response : [8, 5, 7, 6, 4]
LLM's response : [8, 9, 7, 9, 8]
LLM's response : [8, 6, 7, 5, 9]
LLM's response : [8, 5, 7, 6, 9]
LLM's response : [8, 7, 9, 6, 9]
LLM's response : [8, 6, 9, 4, 7]
LLM's response : [9, 7, 8, 7, 10]
best_score :  0.9285714285714286


alt.Chart(...)